In [1]:
import os
from colorama import Fore, Style, Back
from collections import defaultdict

In [150]:
def notebook_IO(notebook_path, 
                ext_list, 
                pattern_list, 
                project_dir, 
                missing_only=False):
    """Takes the path of Ni's notebooks and extracts lines 
    whereby a file is potentially read or written."""
    
    import json
    import re
    from pathlib import Path

    notebook = json.load(open(notebook_path))
    compiled_patt = re.compile(r"(?:{string})".format(string='|'.join(ext_list+pattern_list)), flags=re.IGNORECASE)
    files_patt = re.compile(r"(?:{string})".format(string='|'.join(ext_list)), flags=re.IGNORECASE)
    notebook_cells = notebook['cells']
    path_vars = dict()
    for c_id,c in enumerate(notebook_cells):
        if c['cell_type'] == 'code':
            count = 0
            for i,s in enumerate(c['source']):
                s = re.sub(r"[\r\n\s]+$", '', s)
                if (i>(count) or i==0):
                    count=i
                    if not (s.startswith('#') or '!' in s):
                        if compiled_patt.search(s):
                            if s.endswith(('(', '[', '{')):
                                if s.endswith('('):
                                    open_char = '('
                                    close_char = ')'
                                elif s.endswith('['):
                                    open_char = '['
                                    close_char = ']'
                                else:
                                    open_char = '{'
                                    close_char = '}'
                                while ((s.count(open_char)!=s.count(close_char)) and 
                                       ((s.endswith(close_char)==False) or 
                                        (s.endswith(''.join([close_char, '\n']))==False))):
                                    count+=1
                                    try:
                                        next_str = c['source'][count]
                                        s = ''.join([s, re.sub(r"^[\r\n\s]+|[\t\s]+$", "", next_str)])
                                    except IndexError:
                                        s = ''.join([s, re.sub(r"^[\r\n\s]+|[\t\s]+$", "", next_str)])
                                        break
                            s = re.sub(r"^[\r\n\s]+|[\r\n\s]+$", '', s)
                            if files_patt.search(s):
                                if re.search(r"([^'\"]*.{file_patt})(?=('|\"))".format(file_patt=files_patt.pattern), s):
                                    file = s
                                    if (('{' in file) and ('}' in file)):
                                        file = re.sub(r"{.*?}", "*", file)
                                    file = re.search(r"([^'\"]*.{file_patt})(?=('|\"))".format(file_patt=files_patt.pattern), file).group(0)
                                    file = file.split('../')[-1]
                                    file = file.split('./')[-1]
                                    if file.startswith('/'):
                                        file = file.split('/', 1)[-1]
                                    files_ls = []
                                    for f in Path(project_dir).glob(f"**/{file.split('/')[-1]}"):
                                        files_ls.append(str(f))
                                    files_ls = [fls.replace(project_dir, '...') for fls in files_ls]
                                    files = '\n\t'.join(files_ls)
                                    read_compile = re.compile(r"(?:read)", flags=re.IGNORECASE)
                                    write_compile = ['write', 'file=', 'filename=', 'file_name', 
                                                     'save', '\.to_csv']
                                    write_compile = re.compile(r"(?:{write})".format(write='|'.join(write_compile)), flags=re.IGNORECASE)
                                    
                                    def catch_path_vars(_str):
                                        """Capture variable string saving IO path."""
                                        
                                        patt = re.compile(r"\w.+?(?=(=|\s=|<-|\s<-))", flags=re.IGNORECASE)
                                        if patt.search(_str):
                                            return patt.search(_str).group(0)
                                        else:
                                            return None
                                    
                                    if files_ls:
                                        if missing_only:
                                            continue
                                        if any('/'.join(file.split('/')[:-1]) in i for i in files_ls):
                                            print(f"\n{Fore.LIGHTMAGENTA_EX}Code block index: {c_id} <-------> Execution count: {c['execution_count']}{Style.RESET_ALL}\n")
                                            print(f"{Fore.LIGHTGREEN_EX}Source : {Style.RESET_ALL}" + "\n\t" + \
                                                  f"{Fore.GREEN}{s}{Style.RESET_ALL}" + "\n" + \
                                                  f"{Fore.LIGHTBLUE_EX}Pattern : {Style.RESET_ALL}" + "\n\t" + \
                                                  f"{Fore.BLUE}**/{file.split('/')[-1]}{Style.RESET_ALL}" + "\n" + \
                                                  f"{Fore.LIGHTCYAN_EX}Paths : {Style.RESET_ALL}" + "\n\t" + \
                                                  f"{Fore.CYAN}{files}{Style.RESET_ALL}")
                                            if read_compile.search(s):
                                                yield('present', 'read', f"**/{file.split('/')[-1]}")
                                            elif write_compile.search(s):
                                                yield('present', 'write', f"**/{file.split('/')[-1]}")
                                            else:
                                                if catch_path_vars(s):
                                                    path_vars[catch_path_vars(s)] = ['present', 'unknown', f"**/{file.split('/')[-1]}"]
                                                yield('present', 'unknown', f"**/{file.split('/')[-1]}")
                                        else:
                                            print(f"\n{Fore.LIGHTMAGENTA_EX}Code block index: {c_id} <-------> Execution count: {c['execution_count']}{Style.RESET_ALL}\n")
                                            print(f"{Fore.LIGHTYELLOW_EX}Source : {Style.RESET_ALL}" + "\n\t" + \
                                                  f"{Fore.YELLOW}{s}{Style.RESET_ALL}" + "\n" + \
                                                  f"{Fore.LIGHTBLUE_EX}Pattern : {Style.RESET_ALL}" + "\n\t" + \
                                                  f"{Fore.BLUE}**/{file.split('/')[-1]}{Style.RESET_ALL}" + "\n" + \
                                                  f"{Fore.LIGHTCYAN_EX}Paths : {Style.RESET_ALL}" + "\n\t" + \
                                                  f"{Fore.CYAN}{files}{Style.RESET_ALL}")
                                            if read_compile.search(s):
                                                yield('ambiguous', 'read', f"**/{file.split('/')[-1]}")
                                            elif write_compile.search(s):
                                                yield('ambiguous', 'write', f"**/{file.split('/')[-1]}")
                                            else:
                                                if catch_path_vars(s):
                                                    path_vars[catch_path_vars(s)] = ['ambiguous', 'unknown', f"**/{file.split('/')[-1]}"]
                                                yield('ambiguous', 'unknown', f"**/{file.split('/')[-1]}")
                                    else:
                                        print(f"\n{Fore.LIGHTMAGENTA_EX}Code block index: {c_id} <-------> Execution count: {c['execution_count']}{Style.RESET_ALL}\n")
                                        print(f"{Fore.LIGHTRED_EX}Source : {Style.RESET_ALL}" + "\n\t" + \
                                              f"{Fore.RED}{s}{Style.RESET_ALL}" + "\n" + \
                                              f"{Fore.LIGHTBLUE_EX}Pattern : {Style.RESET_ALL}" + "\n\t" + \
                                              f"{Fore.BLUE}**/{file.split('/')[-1]}{Style.RESET_ALL}")
                                        if read_compile.search(s):
                                            yield('missing', 'read', f"**/{file.split('/')[-1]}")
                                        elif write_compile.search(s):
                                            yield('missing', 'write', f"**/{file.split('/')[-1]}")
                                        else:
                                            if catch_path_vars(s):
                                                path_vars[catch_path_vars(s)] = ['missing', 'unknown', f"**/{file.split('/')[-1]}"]
                                            yield('missing', 'unknown', f"**/{file.split('/')[-1]}")
                            else:
                                if not missing_only:
                                    print(f"\n{Fore.LIGHTMAGENTA_EX}Code block index: {c_id} <-------> Execution count: {c['execution_count']}{Style.RESET_ALL}\n")
                                    print(s)
                                    for k in path_vars.keys():
                                        if re.search(k, s):
                                            if read_compile.search(s):
                                                yield(path_vars[k][0], 'read', path_vars[k][2])
                                            elif write_compile.search(s):
                                                yield(path_vars[k][0], 'write', path_vars[k][2])

In [151]:
code_dirs = ['../code/notebooks/analysis/', '../code/notebooks/figures/']
exts = ['\.h5ad', '\.h5', '\.hdf5', '\.png', '\.pdf', '\.txt', '\.list', 
        '\.csv', '\.tsv', '\.pkl', '\.gz']
patts = ['read', '\.\./', '\./', 'write', 'file=', 'filename=', 'file_name', 'save', '\.to_csv']
project = "/lustre/scratch126/cellgen/team205/nh3/skin"

In [152]:
# import re

# test_patt = re.compile(r"\w.+?(?=(=|\s=|<-|\s<-))", flags=re.IGNORECASE)
# test_str = "obs <- read.csv('../20210611_final_object/fetal_skin.norm.maternal_removed.20220202.obs.csv.gz', row.names=1)"
# test_patt.search(test_str)

In [113]:
%%time

for d in code_dirs:
    print(f"{Back.WHITE}{Fore.BLACK}{d}{''.join([' ' for i in range(int(150-(len(d)/2)))])}\n{Style.RESET_ALL}")
    for n in os.listdir(d):
        if ('.ipynb' in n) and ('checkpoint' not in n):
            print(f"{Fore.MAGENTA}{''.join([' ' for i in range(int(75-(len(n)/2)))])}{n}{Style.RESET_ALL}", "\n")
            for io in notebook_IO(d+n, exts, patts, project, missing_only=True):
                pass
            print(f"{Fore.MAGENTA}    {' '.join(['-' for i in range(70)])}{Style.RESET_ALL}", "\n")

../code/notebooks/analysis/                                                                                                                                        

                                                              2.7_Keratinocytes_v2.ipynb 

    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

                                                              2.2_Myloid_cells_v2.ipynb 

    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

                                                             5.2_make_final_object2.ipynb 

    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

                                                                2.9_Erythroid_v2.ipynb 

    - - - - - - - - - - - - - - - -

In [153]:
%%time

notebooks_IO_dict = defaultdict(dict)
notebooks_IO_dict['type'] = []

for d in code_dirs:
    print(f"{Back.WHITE}{Fore.BLACK}{d}{''.join([' ' for i in range(int(150-(len(d)/2)))])}\n{Style.RESET_ALL}")
    for n in os.listdir(d):
        if ('.ipynb' in n) and ('checkpoint' not in n):
            print(f"{Fore.MAGENTA}{''.join([' ' for i in range(int(75-(len(n)/2)))])}{n}{Style.RESET_ALL}", "\n")
            file_patt_ls = []
            io_list = []
            state_list = []
            for io in notebook_IO(d+n, exts, patts, project):
                if io[2] in file_patt_ls:
                    if not any((io_list[i]==io[1] and e==io[2]) for i,e in enumerate(file_patt_ls)):
                        if (io[1]=='read' or io[1]=='write'):
                            for i,e in enumerate(file_patt_ls):
                                if (io_list[i]=='unknown' and e==io[2]):
                                    del file_patt_ls[i]
                                    del io_list[i]
                                    del state_list[i]
                            state_list.append(io[0])
                            io_list.append(io[1])
                            file_patt_ls.append(io[2])
                        else:
                            if not any(((io_list[i]=='read' or io_list[i]=='write') and e==io[2]) for i,e in enumerate(file_patt_ls)):
                                state_list.append(io[0])
                                io_list.append(io[1])
                                file_patt_ls.append(io[2])
                else:
                    state_list.append(io[0])
                    io_list.append(io[1])
                    file_patt_ls.append(io[2])
            print(f"{Fore.MAGENTA}    {' '.join(['-' for i in range(70)])}{Style.RESET_ALL}", "\n")
            notebooks_IO_dict['type'].append(d)
            notebooks_IO_dict[n]['paths'] = file_patt_ls
            notebooks_IO_dict[n]['IO'] = io_list
            notebooks_IO_dict[n]['state'] = state_list

../code/notebooks/analysis/                                                                                                                                        

                                                              2.7_Keratinocytes_v2.ipynb 


Code block index: 11 <-------> Execution count: 359

Source : 
	fetal_ad = sc.read('data/h5ad/fetal_skin_raw.h5ad', mode='r')
Pattern : 
	**/fetal_skin_raw.h5ad
Paths : 
	.../data/h5ad/fetal_skin_raw.h5ad

Code block index: 16 <-------> Execution count: 362

Source : 
	g1_s = pd.read_csv('data/misc/G1-S_genes.list', header=None, names=['gene'])
Pattern : 
	**/G1-S_genes.list
Paths : 
	.../data/misc/G1-S_genes.list

Code block index: 16 <-------> Execution count: 362

Source : 
	g2_m = pd.read_csv('data/misc/G2-M_genes.list', header=None, names=['gene'])
Pattern : 
	**/G2-M_genes.list
Paths : 
	.../data/misc/G2-M_genes.list

Code block index: 16 <-------> Execution count: 362

Source : 
	jp_cc = pd.read_csv('data/misc/JP_cycle_genes.l

In [154]:
notebooks_IO_dict['0_collectRaw.ipynb']['IO'][notebooks_IO_dict['0_collectRaw.ipynb']['paths'].index("**/*_cell_source.csv")]

'read'

In [158]:
%%time
from pathlib import Path

patt_count = set()
file_count = 0
for k in notebooks_IO_dict.keys():
    if '.ipynb' in k:
        for p in notebooks_IO_dict[k]['paths']:
            patt_count.add(p)
for patt in patt_count:
    for f in Path(project).glob(patt):
        if str(f) != '':
            file_count+=1
print('Total number of patterns:', len(patt_count))
print('Total number of files:', file_count)

Total number of patterns: 185
Total number of files: 507
CPU times: user 4.35 s, sys: 6.52 s, total: 10.9 s
Wall time: 1min 8s


In [211]:
all_nbs = [] 
for d in code_dirs:
    for nb in os.listdir(d):
        if ('.ipynb' in nb) and ('checkpoint' not in nb):
            all_nbs.append(nb)
all_nbs = set(all_nbs)
subgraphs = {'Raw Data pre-Processing' : ['0_collectRaw.ipynb', '1_broadCategory.ipynb'], 
             'Cell-Type-Specific Data Analysis' : ['2.1_Stroma_v2.ipynb', '2.2_Myloid_cells_v2.ipynb', '2.3_T_cells_v2.ipynb', 
                                                   '2.4_B_cells_v2.ipynb', '2.5_Endothelium_v2.ipynb', '2.6_Mast_cells_v2.ipynb', 
                                                   '2.7_Keratinocytes_v2.ipynb', '2.8_Melanocytes_v2.ipynb', '2.9_Erythroid_v2.ipynb'], 
             'Data Post-Processing' : ['3_remove_doublets.ipynb', '4_remake_global_umap.ipynb', '5.1_make_final_object.ipynb', 
                                       '5.2_make_final_object2.ipynb', '5.3_make_final_object3.ipynb'], 
             'pre-processing for figures' : ['integrate_rie20.ipynb', 'Compare_to_Bian20.ipynb'], 
             'figures' : ['figS2L.ipynb', 'Fig2C.ipynb', 'figS2K.ipynb', 'fig1E_20220211.ipynb', 
                          'fig1B_20220202.ipynb', 'fig3.ipynb', 'fig2D.ipynb', 'figS2J.ipynb']}
print(sorted([item for sublist in subgraphs.values() for item in sublist])==sorted(all_nbs))
print(subgraphs.values())
print([item for sublist in subgraphs.values() for item in sublist])
print(sorted(all_nbs))

False
dict_values([['0_collectRaw.ipynb', '1_broadCategory.ipynb'], ['2.1_Stroma_v2.ipynb', '2.2_Myloid_cells_v2.ipynb', '2.3_T_cells_v2.ipynb', '2.4_B_cells_v2.ipynb', '2.5_Endothelium_v2.ipynb', '2.6_Mast_cells_v2.ipynb', '2.7_Keratinocytes_v2.ipynb', '2.8_Melanocytes_v2.ipynb', '2.9_Erythroid_v2.ipynb'], ['3_remove_doublets.ipynb', '4_remake_global_umap.ipynb', '5.1_make_final_object.ipynb', '5.2_make_final_object2.ipynb', '5.3_make_final_object3.ipynb'], ['integrate_rie20.ipynb', 'Compare_to_Bian20.ipynb'], ['figS2L.ipynb', 'Fig2C.ipynb', 'figS2K.ipynb', 'fig1E_20220211.ipynb', 'fig1B_20220202.ipynb', 'fig3.ipynb', 'fig2D.ipynb', 'figS2J.ipynb']])
['0_collectRaw.ipynb', '1_broadCategory.ipynb', '2.1_Stroma_v2.ipynb', '2.2_Myloid_cells_v2.ipynb', '2.3_T_cells_v2.ipynb', '2.4_B_cells_v2.ipynb', '2.5_Endothelium_v2.ipynb', '2.6_Mast_cells_v2.ipynb', '2.7_Keratinocytes_v2.ipynb', '2.8_Melanocytes_v2.ipynb', '2.9_Erythroid_v2.ipynb', '3_remove_doublets.ipynb', '4_remake_global_umap.ipyn

In [212]:
%%time
import string

alph = list(string.ascii_uppercase)
mermaid_dict = defaultdict(dict)
ind = 0
for index,s in enumerate(subgraphs.values()):
    for k in s:
        try:
            mermaid_dict['for_map'][k] = '_'.join([alph[ind], str(index)])
        except IndexError:
            mermaid_dict['for_map'][k] = f"{alph[ind-26]}_".join([alph[ind-26], str(index)])
        mermaid_dict['rev_map'][mermaid_dict['for_map'][k]] = k
        mermaid_dict['edge_to'][mermaid_dict['for_map'][k]] = set()
        mermaid_dict['edge_from'][mermaid_dict['for_map'][k]] = set()
        mermaid_dict['edge_unk'][mermaid_dict['for_map'][k]] = set()
        for i, patt in enumerate(notebooks_IO_dict[k]['paths']):
            if patt not in mermaid_dict['for_map'].keys():
                mermaid_dict['for_map'][patt] = '_'.join([mermaid_dict['for_map'][k], str(i+1), notebooks_IO_dict[k]['state'][i]])
                mermaid_dict['rev_map'][mermaid_dict['for_map'][patt]] = patt
                if notebooks_IO_dict[k]['IO'][i]=='write':
                    mermaid_dict['edge_to'][mermaid_dict['for_map'][k]].add(mermaid_dict['for_map'][patt])
                elif notebooks_IO_dict[k]['IO'][i]=='read':
                    mermaid_dict['edge_from'][mermaid_dict['for_map'][k]].add(mermaid_dict['for_map'][patt])
                else:
                    mermaid_dict['edge_unk'][mermaid_dict['for_map'][k]].add(mermaid_dict['for_map'][patt])
            if notebooks_IO_dict[k]['IO'][i]=='read':
                if patt not in mermaid_dict['edge_from'][mermaid_dict['for_map'][k]]:
                    mermaid_dict['for_map'][patt] = '_'.join([mermaid_dict['for_map'][k], str(i+1), notebooks_IO_dict[k]['state'][i]])
                    mermaid_dict['rev_map'][mermaid_dict['for_map'][patt]] = patt
                    if patt in mermaid_dict['edge_unk'][mermaid_dict['for_map'][k]]:
                        mermaid_dict['edge_unk'][mermaid_dict['for_map'][k]].remove(mermaid_dict['for_map'][patt])
                mermaid_dict['edge_from'][mermaid_dict['for_map'][k]].add(patt)
            elif notebooks_IO_dict[k]['IO'][i]=='write':
                mermaid_dict['edge_to'][mermaid_dict['for_map'][k]].add(patt)
                if patt in mermaid_dict['edge_unk'][mermaid_dict['for_map'][k]]:
                    mermaid_dict['edge_unk'][mermaid_dict['for_map'][k]].remove(patt)
            else:
                if patt not in mermaid_dict['edge_from'][mermaid_dict['for_map'][k]]:
                    if patt not in mermaid_dict['edge_to'][mermaid_dict['for_map'][k]]:
                        print(f"{Fore.RED}{patt}{Style.RESET_ALL}")
                        mermaid_dict['edge_unk'][mermaid_dict['for_map'][k]].add(patt)
        ind+=1

mermaid_dict

**/tmp.h5ad
**/fetal_skin.endothelium.doublet_removed_processed.20200403.h5ad
**/fetal_skin.lymphoid_T_NK.doublet_removed_processed.20200403.h5ad
**/fetal_skin.erythroid.doublet_removed_processed.20200403.h5ad
**/fetal_skin.mast_cells.doublet_removed_processed.20200403.h5ad
**/fetal_skin.keratinocytes.doublet_removed_processed.20200403.h5ad
**/fetal_skin.melanocytes.doublet_removed_processed.20200403.h5ad
**/fetal_skin.lymphoid_B.doublet_removed_processed.20200403.h5ad
**/fetal_skin.myeloid.doublet_removed_processed.20200403.h5ad
**/fetal_skin.muscle_cells_pass2.processed.cellxgene.h5ad
**/fetal_skin.neuronal_cells_pass2.processed.cellxgene.h5ad
**/fetal_skin.fibroblast.doublet_removed_processed_redo.20200605.h5ad
**/fetal_skin.stroma_cells_pass2.processed.annotated.cellxgene.h5ad
**/fetal_skin.*.wilcoxon.deg_top100.tsv
**/pooled_adult_lognorm_LR.pkl
**/pooled_fetal_lognorm_LR.pkl
**/fetal_skin.independent_annotation_refined_20220202.lognorm.pkl
**/figS1G.pdf
**/figS1G_smaller.pdf
**/a

defaultdict(dict,
            {'for_map': {'0_collectRaw.ipynb': 'A_0',
              '**/fetal_skin_samples.txt': 'A_0_1_present',
              '**/*_raw.h5': 'A_0_2_present',
              '**/*_cell_source.csv': 'A_0_3_present',
              '**/knee_plots_per_run.png': 'A_0_4_present',
              '**/fetal_annotation_rachel-201904.txt': 'J_1_5_present',
              '**/fetal_skin_raw.20190926.h5ad': 'K_1_1_present',
              '1_broadCategory.ipynb': 'B_0',
              '**/hierarchy1.txt': 'J_1_6_present',
              '**/tmp.h5ad': 'B_0_3_missing',
              '**/rachel_annot_201904.pca.pkl': 'B_0_4_present',
              '**/fetal_skin_processed.hierarchy1.h5ad': 'B_0_5_missing',
              '**/fetal_skin_bbknn.h5ad': 'B_0_6_present',
              '**/fetal_skin.hierarchy1.*.norm.h5ad': 'B_0_7_missing',
              '2.1_Stroma_v2.ipynb': 'C_1',
              '**/fetal_skin_hierarch1_*.h5ad': 'D_1_3_present',
              '**/2.*_*_harmony_withCC_leiden_r

In [213]:
patt_count.difference(set(mermaid_dict['for_map'].keys()))

{'**/*.hiconf_contig_igblast_db-pass.tsv',
 '**/GSE133341_thymus_lymphoid_processed.h5ad',
 '**/PAN.A01.v01.entire_data_normalised_log.20210429.full_obs.annotated.clean.csv.gz',
 '**/fetal_skin.lymphoid_T_NK.doublet_removed_processed.20200403.TCR.h5ad',
 '**/fetal_skin.norm.maternal_removed.20220202.obs.csv.gz',
 '**/fetal_skin.processed_with_norm_X_counts_layer.maternal_removed.20211029.h5ad',
 '**/fetal_skin.scvi_donor.milo.fig1b_annotation_vs_pcw.20220202.pdf',
 '**/fig3C_candidates.20221017.pdf',
 '**/gdTCR_metadata_02082021.csv',
 '**/lymphoid_for_ni_20191128.h5ad',
 '**/pooled_endothelium.processed.h5ad'}

In [83]:
for i in notebooks_IO_dict.items():
    if '.ipynb' in i[0]:
        print(len(notebooks_IO_dict[i[0]]['paths'])==len(notebooks_IO_dict[i[0]]['IO']))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [163]:
print("""```mermaid
flowchart LR
\t%%{init: {'securityLevel' : 'loose', 'flowchart': { 'curve': 'basis' }, 'themeVariables': {'fontSize': '55px', 'fontFamily': 'Inter', 'clusterBkg':'#FBD15F', 'clusterBorder':'#5FFBD1'}}}%%
\tlinkStyle default stroke-width:4px
\tclassDef green color:#022e1f,fill:#00f500,stroke-width:4px
\tclassDef red color:#022e1f,fill:#FA7441,stroke-width:4px
\tclassDef blue color:#022e1f,fill:#6EC7F7,stroke-width:4px
\tclassDef grey color:#022e1f,fill:#C9BFC5,stroke-width:4px
\tclassDef black color:#fff,fill:#000,stroke-width:4px\n""")

```mermaid
flowchart LR
	%%{init: {'securityLevel' : 'loose', 'flowchart': { 'curve': 'basis' }, 'themeVariables': {'fontSize': '55px', 'fontFamily': 'Inter', 'clusterBkg':'#FBD15F', 'clusterBorder':'#5FFBD1'}}}%%
	linkStyle default stroke-width:4px
	classDef green color:#022e1f,fill:#00f500,stroke-width:4px
	classDef red color:#022e1f,fill:#FA7441,stroke-width:4px
	classDef blue color:#022e1f,fill:#6EC7F7,stroke-width:4px
	classDef grey color:#022e1f,fill:#C9BFC5,stroke-width:4px
	classDef black color:#fff,fill:#000,stroke-width:4px



In [185]:
mermaid_str = ["""```mermaid
flowchart LR
\t%%{init: {'securityLevel' : 'loose', 'flowchart': { 'curve': 'basis' }, 'themeVariables': {'nodeBorder':'#837E6D', 'fontSize': '55px', 'fontFamily': 'Inter', 'clusterBkg':'#FBD15F', 'clusterBorder':'#5FFBD1'}}}%%
\tlinkStyle default stroke-width:8px\n"""]

colours = {'red' : {'color':'#000',
                    'fill':'#FF6041'},
           'anti_red_0' : {'color':'#000',
                           'fill':'#94FF3F'}, 
           'anti_red_1' : {'color':'#000',
                           'fill':'#3FF4FF'}, 
           'anti_red_2' : {'color':'#FFF',
                           'fill':'#AA3FFF'}, 
           'blue' : {'color':'#FFF',
                     'fill':'#002190'}, 
           'anti_blue_0' : {'color':'#FFF',
                            'fill':'#900069'}, 
           'anti_blue_1' : {'color':'#000',
                            'fill':'#906F00'}, 
           'anti_blue_2' : {'color':'#000',
                            'fill':'#009027'}, 
           'mustard' : {'color':'#000',
                      'fill':'#9CA141'}, 
           'anti_mustard_0' : {'color':'#000',
                            'fill':'#41A176'}, 
           'anti_mustard_1' : {'color':'#000',
                            'fill':'#4641A1'}, 
           'anti_mustard_2' : {'color':'#000',
                            'fill':'#A1416C'}}

for col in colours:
    mermaid_str.append(f"\tclassDef {col} color:{colours[col]['color']},fill:{colours[col]['fill']},stroke-width:5px\n")

mermaid_str.append("\tclassDef black color:#fff,fill:#000,stroke-width:5px\n")

print(''.join(mermaid_str))

```mermaid
flowchart LR
	%%{init: {'securityLevel' : 'loose', 'flowchart': { 'curve': 'basis' }, 'themeVariables': {'nodeBorder':'#837E6D', 'fontSize': '55px', 'fontFamily': 'Inter', 'clusterBkg':'#FBD15F', 'clusterBorder':'#5FFBD1'}}}%%
	linkStyle default stroke-width:8px
	classDef red color:#000,fill:#FF6041,stroke-width:5px
	classDef anti_red_0 color:#000,fill:#94FF3F,stroke-width:5px
	classDef anti_red_1 color:#000,fill:#3FF4FF,stroke-width:5px
	classDef anti_red_2 color:#FFF,fill:#AA3FFF,stroke-width:5px
	classDef blue color:#FFF,fill:#002190,stroke-width:5px
	classDef anti_blue_0 color:#FFF,fill:#900069,stroke-width:5px
	classDef anti_blue_1 color:#000,fill:#906F00,stroke-width:5px
	classDef anti_blue_2 color:#000,fill:#009027,stroke-width:5px
	classDef mustard color:#000,fill:#9CA141,stroke-width:5px
	classDef anti_mustard_0 color:#000,fill:#41A176,stroke-width:5px
	classDef anti_mustard_1 color:#000,fill:#4641A1,stroke-width:5px
	classDef anti_mustard_2 color:#000,fill:#A1416C,

In [219]:
mermaid_str = ["""```mermaid
flowchart LR
\t%%{init: {'securityLevel' : 'loose', 'flowchart': {'curve': 'basis', 'rankSpacing':1, 'nodeSpacing':10}, 'themeVariables': {'nodeBorder':'#837E6D', 'fontSize': '150px', 'fontFamily': 'Inter', 'clusterBkg':'#FBD15F', 'clusterBorder':'#5FFBD1'}}}%%
\tlinkStyle default stroke-width:8px\n"""]

colours = {'red' : {'color':'#000',
                    'fill':'#FF6041'},
           'anti_red_0' : {'color':'#000',
                           'fill':'#94FF3F'}, 
           'anti_red_1' : {'color':'#000',
                           'fill':'#3FF4FF'}, 
           'anti_red_2' : {'color':'#FFF',
                           'fill':'#AA3FFF'}, 
           'blue' : {'color':'#FFF',
                     'fill':'#002190'}, 
           'anti_blue_0' : {'color':'#FFF',
                            'fill':'#900069'}, 
           'anti_blue_1' : {'color':'#000',
                            'fill':'#906F00'}, 
           'anti_blue_2' : {'color':'#000',
                            'fill':'#009027'}, 
           'mustard' : {'color':'#000',
                      'fill':'#9CA141'}, 
           'anti_mustard_0' : {'color':'#000',
                            'fill':'#41A176'}, 
           'anti_mustard_1' : {'color':'#000',
                            'fill':'#4641A1'}, 
           'anti_mustard_2' : {'color':'#000',
                            'fill':'#A1416C'}}

for col in colours:
    mermaid_str.append(f"\tclassDef {col} color:{colours[col]['color']},fill:{colours[col]['fill']},stroke-width:8px\n")

mermaid_str.append("\tclassDef black color:#fff,fill:#000,stroke-width:8px\n")

git_repo_dir = '/home/jovyan/FCA_Fetal_Skin_priv'

edge_count = 0

for subg,comp in subgraphs.items():
    mermaid_str.append(f"\tsubgraph {subg}\n")
    for nb_name in comp:
        nb_revname = mermaid_dict['for_map'][nb_name]
        mermaid_str.append(f"\t\t{nb_revname}>{nb_name}]:::red\n")
        for p in Path(git_repo_dir).glob(f"**/{nb_name}"):
            if p:
                mermaid_str.append(f"\t\tclick {nb_revname} \"https://github.com/haniffalab/FCA_Fetal_Skin_priv/blob/main{str(p).split(git_repo_dir)[-1]}\"\n")
                del p
                break
        for f in mermaid_dict['edge_from'][nb_revname]:
            for p in Path(git_repo_dir).glob(f):
                if p:
                    mermaid_str.append(f"\t\tclick {mermaid_dict['for_map'][f]} \"https://github.com/haniffalab/FCA_Fetal_Skin_priv/blob/main{str(p).split(git_repo_dir)[-1]}\"\n")
                    del p
                    break
            if 'missing' in mermaid_dict['for_map'][f]:
                continue
#                 mermaid_str.append(f"\t\t{mermaid_dict['for_map'][f]}[({f})]:::black\n")
            elif '.h5ad' in f:
                mermaid_str.append(f"\t\t{mermaid_dict['for_map'][f]}[({f})]:::anti_red_0\n")
            else:
                mermaid_str.append(f"\t\t{mermaid_dict['for_map'][f]}[({f})]:::anti_red_1\n")
        for f in mermaid_dict['edge_to'][nb_revname]:
            if f not in mermaid_dict['edge_from'][nb_revname]:
                for p in Path(git_repo_dir).glob(f):
                    if p:
                        mermaid_str.append(f"\t\tclick {mermaid_dict['for_map'][f]} \"https://github.com/haniffalab/FCA_Fetal_Skin_priv/blob/main{str(p).split(git_repo_dir)[-1]}\"\n")
                        del p
                        break
                if 'missing' in mermaid_dict['for_map'][f]:
                    continue
#                     mermaid_str.append(f"\t\t{mermaid_dict['for_map'][f]}[({f})]:::black\n")
                elif '.h5ad' in f:
                    mermaid_str.append(f"\t\t{mermaid_dict['for_map'][f]}[({f})]:::anti_red_0\n")
                else:
                    mermaid_str.append(f"\t\t{mermaid_dict['for_map'][f]}[({f})]:::anti_red_1\n")
#         for f in mermaid_dict['edge_unk'][nb_revname]:
#             for p in Path(git_repo_dir).glob(f):
#                 if p:
#                     mermaid_str.append(f"\t\tclick {mermaid_dict['for_map'][f]} \"https://github.com/haniffalab/FCA_Fetal_Skin_priv/blob/main{str(p).split(git_repo_dir)[-1]}\"\n")
#                     del p
#                     break
#             if 'missing' in mermaid_dict['for_map'][f]:
#                 continue
# #                 mermaid_str.append(f"\t\t{mermaid_dict['for_map'][f]}[({f})]:::black\n")
#             elif '.h5ad' in f:
#                 mermaid_str.append(f"\t\t{mermaid_dict['for_map'][f]}[({f})]:::anti_red_0\n")
#             else:
#                 mermaid_str.append(f"\t\t{mermaid_dict['for_map'][f]}[({f})]:::anti_red_1\n")
    mermaid_str.append(f"\tend\n")
col_count = 0
for nb_node, io_nodes in mermaid_dict['edge_from'].items():
    for file in io_nodes:
        if 'missing' in mermaid_dict['for_map'][file]:
            continue
        if col_count < (len(colours)/2)-1:
            col_count += 1
        else:
            col_count = 0
        mermaid_str.append(f"\t{mermaid_dict['for_map'][file]} --> {nb_node}\n")
        mermaid_str.append(f"\tlinkStyle {edge_count} stroke:{colours[list(colours.keys())[col_count]]['fill']}, stroke-width:18px\n")
        edge_count += 1

for nb_node, io_nodes in mermaid_dict['edge_to'].items():
    for file in io_nodes:
        if 'missing' in mermaid_dict['for_map'][file]:
            continue
        if col_count <= int(len(colours)/2)-1:
            col_count = int(len(colours)/2)
        elif col_count < len(colours)-1:
            col_count += 1
        else:
            col_count = int(len(colours)/2)
        mermaid_str.append(f"\t{nb_node} --> {mermaid_dict['for_map'][file]}\n")
        mermaid_str.append(f"\tlinkStyle {edge_count} stroke:{colours[list(colours.keys())[col_count]]['fill']}, stroke-width:18px\n")
        edge_count += 1

mermaid_str.append("```")

with open("../test_mermaid_copy.md", mode='w') as mermaid_md:
    mermaid_md.write(''.join(mermaid_str))


print(''.join(mermaid_str))


```mermaid
flowchart LR
	%%{init: {'securityLevel' : 'loose', 'flowchart': {'curve': 'basis', 'rankSpacing':1, 'nodeSpacing':10}, 'themeVariables': {'nodeBorder':'#837E6D', 'fontSize': '150px', 'fontFamily': 'Inter', 'clusterBkg':'#FBD15F', 'clusterBorder':'#5FFBD1'}}}%%
	linkStyle default stroke-width:8px
	classDef red color:#000,fill:#FF6041,stroke-width:8px
	classDef anti_red_0 color:#000,fill:#94FF3F,stroke-width:8px
	classDef anti_red_1 color:#000,fill:#3FF4FF,stroke-width:8px
	classDef anti_red_2 color:#FFF,fill:#AA3FFF,stroke-width:8px
	classDef blue color:#FFF,fill:#002190,stroke-width:8px
	classDef anti_blue_0 color:#FFF,fill:#900069,stroke-width:8px
	classDef anti_blue_1 color:#000,fill:#906F00,stroke-width:8px
	classDef anti_blue_2 color:#000,fill:#009027,stroke-width:8px
	classDef mustard color:#000,fill:#9CA141,stroke-width:8px
	classDef anti_mustard_0 color:#000,fill:#41A176,stroke-width:8px
	classDef anti_mustard_1 color:#000,fill:#4641A1,stroke-width:8px
	classDef anti_

In [196]:
len(list(colours.keys()))

12

In [104]:
for pkl in Path(project).glob('**/*.pkl'):
    if (pkl.stat().st_size)/10e3 < 4:
        print(f"{'.../'.join(str(pkl).split('/lustre/scratch126/cellgen/team205/nh3/skin/'))} :", 
              f"{Fore.RED}{(pkl.stat().st_size)/10e3}{Style.RESET_ALL}")

.../notebooks/rachel_annot_201904.pca.pkl : 1.0578
.../20211022_final_figures/LR_models/organoid.pooled_fsk_adt_org.pca_projection.LR.pkl : 0.2927
.../20211022_final_figures/LR_models/fetal.pooled_fsk_adt_org.pca_projection.LR.pkl : 0.5091
.../20211022_final_figures/LR_models/adult.pooled_fsk_ask_org.pca_projection.LR.pkl : 0.3616
.../20200804_fig1/lr_model/pooled_adult_pseudobulk_pca_hm_LR.pkl : 1.3061
.../20200804_fig1/lr_model/pooled_fatel_pca_hm_LR.pkl : 1.1543
.../20200804_fig1/lr_model/pooled_adult_pca_hm_LR.pkl : 1.3256
.../20200804_fig1/lr_model/pooled_fetal_pseudobulk_pca_hm_LR.pkl : 2.4163
